In [9]:
using System;
using System.Collections.Generic;
using System.Linq;

// Определение перечисления для статусов заказа
public enum OrderStatus
{
    Создан,
    Работаем,
    Завершен
}

// Класс для представления заказчика
public class Customer
{
    public string FirstName { get; set; }
    public string LastName { get; set; }

    public Customer(string firstName, string lastName)
    {
        FirstName = firstName;
        LastName = lastName;
    }
}

// Класс для представления сотрудника
public class Employee
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string Position { get; set; }
    public int OrdersProcessed { get; private set; }

    public Employee(string firstName, string lastName, string position)
    {
        FirstName = firstName;
        LastName = lastName;
        Position = position;
        OrdersProcessed = 0;
    }

    // Метод для обработки заказа и увеличения счетчика обработанных заказов
    public void ProcessOrder(Order order)
    {
        OrdersProcessed++;
    }

    // Метод для получения количества обработанных заказов
    public int GetProcessedOrdersCount()
    {
        return OrdersProcessed;
    }
}

// Класс для представления заказа
public class Order
{
    public int OrderId { get; private set; }
    public string Description { get; set; }
    public Customer Customer { get; set; }
    public Employee AssignedEmployee { get; set; }
    public OrderStatus Status { get; set; }
    public DateTime CreatedAt { get; set; }

    private static int _orderIdCounter = 0;

    public Order(string description, Customer customer)
    {
        OrderId = ++_orderIdCounter;
        Description = description;
        Customer = customer;
        Status = OrderStatus.Создан;
        CreatedAt = DateTime.Now;
    }

    // Переопределение метода ToString для удобного вывода информации о заказе
    public override string ToString()
    {
        return $"Order {OrderId}: {Description} - {Customer.FirstName} {Customer.LastName} - {Status}";
    }
}

// Класс для аргументов события изменения статуса заказа
public class OrderStatusChangedEventArgs : EventArgs
{
    public Order Order { get; set; }
    public OrderStatus NewStatus { get; set; }

    public OrderStatusChangedEventArgs(Order order, OrderStatus newStatus)
    {
        Order = order;
        NewStatus = newStatus;
    }
}

// Класс для представления компании, управляющей заказами и сотрудниками
public class Company
{
    public string Name { get; set; }
    private List<Employee> _employees = new List<Employee>();
    private List<Order> _orders = new List<Order>();

    // Определение события изменения статуса заказа
    public event EventHandler<OrderStatusChangedEventArgs> OrderStatusChanged;

    public Company(string name)
    {
        Name = name;
    }

    // Метод для добавления сотрудника в компанию
    public void AddEmployee(Employee employee)
    {
        _employees.Add(employee);
    }

    // Метод для добавления заказа в компанию
    public void AddOrder(Order order)
    {
        _orders.Add(order);
    }

    // Метод для назначения сотрудника на заказ
    public void AssignEmployeeToOrder(Order order, Employee employee)
    {
        order.AssignedEmployee = employee;
        employee.ProcessOrder(order);
    }

    // Метод для изменения статуса заказа и вызова события
    public void ChangeOrderStatus(Order order, OrderStatus newStatus)
    {
        if (order.Status != newStatus)
        {
            order.Status = newStatus;
            OnOrderStatusChanged(order, newStatus);
        }
    }

    // Метод для получения заказов по статусу
    public List<Order> GetOrdersByStatus(OrderStatus status)
    {
        return _orders.Where(o => o.Status == status).ToList();
    }

    // Метод для поиска заказов по заказчику
    public List<Order> FindOrdersByCustomer(Customer customer)
    {
        return _orders.Where(o => o.Customer.FirstName == customer.FirstName && o.Customer.LastName == customer.LastName).ToList();
    }

    // Метод для генерации отчета по сотрудникам
    public void GenerateEmployeeReport()
    {
        foreach (var employee in _employees)
        {
            Console.WriteLine($"{employee.FirstName} {employee.LastName} - Обработанные заказы: {employee.OrdersProcessed}");
        }
    }

    // Защищенный метод для вызова события изменения статуса заказа
    protected virtual void OnOrderStatusChanged(Order order, OrderStatus newStatus)
    {
        OrderStatusChanged?.Invoke(this, new OrderStatusChangedEventArgs(order, newStatus));
    }
}

// Создание компании
Company company = new Company("ФоткиПаукаУМеняНаСтоле Company");

// Создание заказчиков
Customer customer1 = new Customer("John", "Travoltra");
Customer customer2 = new Customer("Jonah", "J. Jameson");

// Создание сотрудников
Employee employee1 = new Employee("Alice", "Madness", "Менеджер");
Employee employee2 = new Employee("Bob", "Killer", "Помощник");

// Добавление сотрудников в компанию
company.AddEmployee(employee1);
company.AddEmployee(employee2);

// Создание заказов
Order order1 = new Order("Описание заказа 1", customer1);
Order order2 = new Order("Описание заказа 2", customer2);
Order order3 = new Order("Описание заказа 3", customer1);

// Добавление заказов в компанию
company.AddOrder(order1);
company.AddOrder(order2);
company.AddOrder(order3);

// Назначение сотрудников на заказы
company.AssignEmployeeToOrder(order1, employee1);
company.AssignEmployeeToOrder(order2, employee2);
company.AssignEmployeeToOrder(order3, employee1);

// Подписка на событие изменения статуса заказа
company.OrderStatusChanged += (sender, args) =>
{
    if (args.Order.AssignedEmployee == employee1)
    {
        Console.WriteLine($"{employee1.FirstName} {employee1.LastName} - Заказ {args.Order.OrderId} статус изменен на {args.NewStatus}");
    }
    else if (args.Order.AssignedEmployee == employee2)
    {
        Console.WriteLine($"{employee2.FirstName} {employee2.LastName} - Заказ {args.Order.OrderId} статус изменен на {args.NewStatus}");
    }
};

// Изменение статусов заказов
Console.WriteLine("\n Изменение статусов заказов:");
company.ChangeOrderStatus(order1, OrderStatus.Создан);
company.ChangeOrderStatus(order2, OrderStatus.Работаем);
company.ChangeOrderStatus(order3, OrderStatus.Завершен);

// Получение заказов по статусу
Console.WriteLine("\n Заказ в процессе:");
foreach (var order in company.GetOrdersByStatus(OrderStatus.Работаем))
{
    Console.WriteLine(order.ToString());
}

// Поиск заказов по заказчику
Console.WriteLine("\n Заказ для John Travoltra:");
foreach (var order in company.FindOrdersByCustomer(customer1))
{
    Console.WriteLine(order.ToString());
}

// Генерация отчета по сотрудникам
Console.WriteLine("\n Отчет сотрудника:");
company.GenerateEmployeeReport();


 Изменение статусов заказов:
Bob Killer - Заказ 2 статус изменен на Работаем
Alice Madness - Заказ 3 статус изменен на Завершен

 Заказ в процессе:
Order 2: Описание заказа 2 - Jonah J. Jameson - Работаем

 Заказ для John Travoltra:
Order 1: Описание заказа 1 - John Travoltra - Создан
Order 3: Описание заказа 3 - John Travoltra - Завершен

 Отчет сотрудника:
Alice Madness - Обработанные заказы: 2
Bob Killer - Обработанные заказы: 1
